# AI & LLM Governance Primer  
**Comprehensive Reference & Lab Notebook**  
*Last updated: 2025-07-09*  

---

This Colab notebook is a **deep‑dive resource** covering the **policies, standards, and practical tooling** that currently shape responsible AI and large‑language‑model (LLM) deployment (2025).

It blends:

* **Curated background reading** (with links & citations)
* **Hands‑on code cells**: checklists, PDF parsing, compliance templates, automated policy lookups
* **Interactive exercises** so learners can apply governance concepts to their own AI projects


## 🎯 Learning Objectives
By the end, you should be able to:

1. **Describe** major AI governance instruments (EU AI Act, NIST AI RMF, ISO 42001, U.S. Executive orders, UK AI Safety Institute, OECD & UNESCO principles).  
2. **Map** LLM use‑cases to risk tiers and compliance obligations.  
3. **Generate** a lightweight *System Card* and *Model Card* using structured prompts.  
4. **Automate** governance tasks: parse policy PDFs, track regulatory updates, run checklists.  
5. **Critically discuss** gaps, emerging trends, and open questions in frontier model oversight.


## 🌍 1. Global Policy Landscape (2024‑2025)

| Region / Org | Instrument | Status | Key Features |
|--------------|------------|--------|--------------|
| **EU** | **AI Act** (Regulation (EU) 2024/1112) | Final text adopted 2024; enforcement starts Aug 2025 (phased) | Risk‑based tiers (unacceptable ⇢ minimal), GPAI obligations, transparency, fundamental‑rights impact assessment |
| **USA (Federal)** | **Executive Order 14110** (Safe, Secure & Trustworthy AI) + **OMB M‑24‑10** memorandum | In force (2023‑2024) | Agency risk management, red‑team sharing, NIST testing, watermarking R&D |
| **USA (NIST)** | **AI Risk Management Framework 1.0** | Voluntary (Jan 2023) | *Govern – Map – Measure – Manage* core functions; profiles & playbooks |
| **ISO/IEC** | **42001:2023 AIMS** | Published Dec 2023 | Management‑system standard akin to ISO 27001 but for AI governance |
| **UK** | **AI Safety Institute** evaluations + *International AI Safety Report 2025* | Ongoing | Frontier‑model testing protocols, red‑team hubs |
| **OECD / UNESCO** | **OECD AI Principles (2019)**, **UNESCO Recommendation (2021)** | Soft law | Human‑centered, accountability, transparency, sustainable society |

> 📌 **Why multiple layers?** Regulations set *obligations*, standards provide *how*, frameworks guide *process*.


## 🔎 2. Map‑Your‑Use‑Case to Risk Tiers  

Run the cell below to input a short description of an AI application and receive:

* EU AI Act tier suggestion  
* Recommended NIST AI RMF controls  
* Flagged ISO 42001 clauses  


In [ ]:
import json, textwrap
description = "Chatbot doctor that gives medical diagnosis to patients online"
print("Use‑case:", description)

# Very lightweight heuristics for demo purposes
tier = "High‑Risk (Annex III – AI in healthcare diagnostics)"
nist_controls = ["Govern.1 Policy", "Map.2 Contextualization", "Measure.3 Bias testing", "Manage.2 Incident response"]
iso_clauses = ["4.2 Understanding the organization and its context",
               "6.1 Actions to address risks and opportunities",
               "8.2 Operational planning and control"]

print("\nEU AI Act tier →", tier)
print("\nNIST AI RMF controls to focus on:")
for c in nist_controls: print(" •", c)
print("\nISO/IEC 42001 clauses:")
for c in iso_clauses: print(" •", c)

> **Pro‑tip**: Replace the heuristic with a call to an LLM:
```python
import openai, os
openai.api_key = os.getenv("OPENAI_API_KEY")
```
Then prompt the model to classify the use‑case via the EU AI Act risk taxonomy.

## 📄 3. Parse Official Documents Programmatically  

The next cell downloads the NIST AI RMF PDF and extracts the *core functions table* so you can filter or embed it in dashboards.


In [ ]:
import requests, io, pdfplumber, re, pandas as pd, textwrap, os, pathlib, json, wget, tempfile, bs4, warnings
url = "https://nvlpubs.nist.gov/nistpubs/ai/nist.ai.100-1.pdf"
local = "/mnt/data/ai_rmf.pdf"
if not pathlib.Path(local).exists():
    wget.download(url, local)

rows = []
with pdfplumber.open(local) as pdf:
    for page in pdf.pages:
        txt = page.extract_text()
        if "Govern" in txt and "Manage" in txt and "Core Function" in txt:
            for line in txt.splitlines():
                m = re.match(r"(Govern|Map|Measure|Manage)\s+(.*)", line)
                if m:
                    rows.append({"Function":m.group(1), "Description":m.group(2)})
df = pd.DataFrame(rows).drop_duplicates()
df

## 📑 4. Generate a Mini *System Card*  

System / model cards document capabilities, limitations & mitigations.  
Fill the template with an LLM for your own model.


In [ ]:
from jinja2 import Template, StrictUndefined
template = Template("""## System Card – {{ model_name }}

### 1. Intended Use
{{ intended_use }}

### 2. Capabilities
{{ capabilities }}

### 3. Limitations & Known Issues
{{ limitations }}

### 4. Safety Evaluations Performed
{{ evals }}

### 5. Alignment & Mitigations
{{ mitigations }}

### 6. Update & Contact
Last updated: {{ today }}.
Contact: {{ contact }}
""", undefined=StrictUndefined)

card = template.render(
    model_name="Student‑Demo LLaMA‑7B",
    intended_use="Educational Q&A for software engineers.",
    capabilities="- Programming help
- Explanation of CS concepts",
    limitations="- May hallucinate stack traces
- Limited knowledge after 2025‑03",
    evals="- TruthfulQA (@62 %)
- Bias & toxicity evaluated with LMHarness.",
    mitigations="- Prompt isolation
- Output filtering via ReACT validation",
    today=str(today),
    contact="gov‑officer@example.com"
)

print(card)

## 📰 5. Governance News Watcher  

Use the snippet below to pull the latest 5 news posts tagged *AI Act* from Reuters RSS for classroom discussion.


In [ ]:
import feedparser, datetime, pytz
feed = feedparser.parse("https://feeds.reuters.com/reuters/technologyNews")
count = 0
for entry in feed.entries:
    if "AI Act" in entry.title and count < 5:
        print(entry.published, "-", entry.title)
        print(entry.link, "\n")
        count +=1

## 💬 6. Discussion & Reflection  

1. **Compliance vs. Innovation:** How can start‑ups comply without stifling creativity?  
2. **Global Fragmentation:** What risks arise from divergent regional rules?  
3. **Enforcement:** Which agencies possess the expertise to audit frontier models?  
4. **Open‑Source Models:** Should open weights be treated differently from proprietary APIs?  


## 📚 Further Reading  

* EU AI Act high‑level summary – artificialintelligenceact.eu  
* NIST AI RMF 1.0 full PDF – nvlpubs.nist.gov/nistpubs/ai/nist.ai.100‑1.pdf  
* ISO/IEC 42001 overview – iso.org/standard/42001  
* White House EO 14110 and OMB M‑24‑10 memo  
* International AI Safety Report 2025 – GOV.UK  
* OpenAI GPT‑4o System Card  

---

© 2025 Prompt Engineering Class Materials – CC‑BY‑SA
